# CSE688: Practical and Application of Machine Learning - Spring 2021
## Assignment 3-2
### Authors

- B073040018 朱劭璿
- B072010029 陳居廷

#### Sentiment analysis on the sentences in the attachment `hw3-2-dataset`. Classify the sentences into two categories: possitive (1) amd negative (0)